Скрипт для классификации отзывов по продуктам, к которым они относятся.

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans

In [4]:
df = pd.read_csv('data/class/train_data.csv')
df.head()

,Unnamed: 0,type,title,text
0,0,Кредит,Просьба об отмене штрафа КАСКО,Здравствуйте! 10.06.2016г. мною был заключен д...
1,1,Ипотека,"В целом хорошо, работали по ипотеке.","Брала ипотеку в юникоре, порадовали квалифика..."
2,2,Реструктуризация,Нас всегда выручает банк Уралсиб!,Знакомство с банком началось с ипотеки в 2009 ...
3,3,Вклад,Абсолютно неприемлемая ситуация и ее решение,Клиент Совкомбанка с 2017 года. В феврале 2018...
4,4,Кредит,Внёс не туда,Пришёл оплачивать кредит и ошибочно внёс деньг...


In [5]:
df['type'] = df['type'].map({'Ипотека': 0, 'Кредит': 1, 'Реструктуризация': 2, 'Вклад': 3,
                             'Дебетовая карта': 4, 'Обслуживание физ. и юр. лиц': 5,
                             'Денежные переводы':6, 'Бизнес услуги': 7, 'Инвестиционные продукты': 8})

Для повышения точности модели, к предикторам были добавлены заголовки отзывов.

In [ ]:
X,Y = ([x1+" "+x2 for (x1, x2) in zip(df['title'], df['text'])], df['type'])

In [ ]:
x_train = X[:8000]
x_test = X[8000:]
y_train = Y[:8000]
y_test = Y[8000:]

Наилучшие результаты получились на данных, обработанных TF-IDF векторайзером без предварительной предобработки текстов.

In [ ]:
tv = TfidfVectorizer()
tv.fit(x_train)

In [ ]:
X_train = tv.transform(x_train)
X_test = tv.transform(x_test)

Определяем модели. Лучший результат получился на бэггинге логистических регрессий.

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(alpha=0.00003, random_state=10, class_weight='balanced', penalty='l1', loss='log')

In [ ]:
from sklearn.ensemble import BaggingClassifier
bsg = BaggingClassifier(sgd, n_jobs=-1, n_estimators=200, random_state=67)
bsg.fit(X_train, y_train)

In [ ]:
print('Acc train: ',bsg.score(X_train, y_train)," Acc test: ", bsg.score(X_test, y_test))

In [ ]:
import pickle
with open('tv.pickle', 'wb') as f:
    """Выгружаем токинайзер"""
    pickle.dump(tv, f)

In [ ]:
import pickle
with open('bsg.pickle', 'wb') as f:
    """Выгружаем модель"""
    pickle.dump(bsg, f)

Загружаем тестовую выборку и делаем предсказания

In [ ]:
df_test = pd.read_csv('data/class/test_data.csv')
x_test = [x1+" "+x2 for (x1, x2) in zip(df_test['title'], df_test['text'])]

In [ ]:
X_test = tv.transform(x_test)
predicts = bsg.predict(X_test)

In [ ]:
predicts = pd.Series(predicts)

In [ ]:
predicts = predicts.map({0: 'Ипотека', 1: 'Кредит', 2: 'Реструктуризация', 
                         3: 'Вклад', 4:'Дебетовая карта', 5:'Обслуживание физ. и юр. лиц',
                         6:'Денежные переводы', 7:'Бизнес услуги', 8: 'Инвестиционные продукты'})

In [ ]:
sub = pd.DataFrame({'index': range(0, len(predicts)),
                    'type':predicts})
sub.to_csv('ranking_answer_class.csv', index=False)